# Implied Scratchpad

Wherein we mess with the "implies" function in the LNN library.  Inspired by [this test file](https://github.com/IBM/LNN/blob/master/tests/learning/propositional/test_bool_implies_1.py).  Since I don't know the language of the FOL community that well, I'm just gonna break shit.

A video on [LNN-EL](https://aclanthology.org/2021.acl-long.64.mp4)

In [33]:
from lnn import Model, Implies, Proposition, World, Direction, Fact, Loss, Predicates, Predicate

TRUE = Fact.TRUE
FALSE = Fact.FALSE
UNKNOWN = Fact.UNKNOWN

I need a deeper understanding of how `world.AXIOM` affects the logic.  This is because my FOL chops aren't what they need to be.  From the docs `world`

* AXIOM – Formulae that follow assumptions of universally being TRUE
* CONTRADICTION – Formulae that are in a contradictory state
* CLOSED – Formulae that follow the closed world assumption
* FALSE – Alias for CLOSED
* OPEN – Formulae that follow the open world assumption

In [55]:
model = Model()
LHS = Proposition("LHS")
RHS = Proposition("RHS")
# Changing this from World.AXIOM to World.CLOSED makes AB True -> False
AB = Implies(LHS, RHS, world=World.AXIOM) # AB -> TRUE
#AB = Implies(LHS, RHS, world=World.OPEN)  # AB -> FALSE
#AB = Implies(LHS, RHS, world=World.CLOSED) # AB -> FALSE
model.add_knowledge(AB)
model.add_data({LHS: TRUE, RHS: FALSE})
model.train(direction=Direction.UPWARD, losses=[Loss.CONTRADICTION])
model.print(params=True)
bias = AB.params("bias")
bounds = AB.state()



***************************************************************************
                                LNN Model

AXIOM Implies: (LHS → RHS)                                  TRUE (1.0, 1.0)
params  α: 1.0,  β: 0.0,  w: [1. 1.]
OPEN Proposition: RHS                                      FALSE (0.0, 0.0)
params  α: 1.0
OPEN Proposition: LHS                                       TRUE (1.0, 1.0)
params  α: 1.0
***************************************************************************


In [41]:
# Sanity check, RHS is now TRUE and now LHS, RHS are Predicates
model = Model()
LHS = Predicate("LHS")
RHS = Predicate("RHS")
#AB = Implies(LHS, RHS, world=World.AXIOM) # AB -> TRUE
AB = Implies(LHS, RHS, world=World.OPEN)  # AB -> TRUE
#AB = Implies(LHS, RHS, world=World.CLOSED) # AB -> FALSE
model.add_knowledge(AB)
model.add_data({
    LHS: {'x': TRUE, 'x2': TRUE},
    RHS: {'y': TRUE, 'y2': (0.1, 0.4)}
})
model.train(direction=Direction.UPWARD, losses=[Loss.CONTRADICTION])
model.print(params=True)
bias = AB.params("bias")


***************************************************************************
                                LNN Model

OPEN Implies: (LHS(0) → RHS(1)) 
params  α: 1.0,  β: 1.0,  w: [1. 1.]
('x', 'y')                                                  TRUE (1.0, 1.0)
('x', 'y2')                                         APPROX_FALSE (0.1, 0.4)
('x2', 'y2')                                        APPROX_FALSE (0.1, 0.4)
('x2', 'y')                                                 TRUE (1.0, 1.0)

OPEN Predicate: RHS 
params  α: 1.0
'y'                                                         TRUE (1.0, 1.0)
'y2'                                                APPROX_FALSE (0.1, 0.4)

OPEN Predicate: LHS 
params  α: 1.0
'x'                                                         TRUE (1.0, 1.0)
'x2'                                                        TRUE (1.0, 1.0)

***************************************************************************


## Get a Weight?

Okay, I'm definitely not using this correctly.  I'm trying to find the general "truthiness" of the rule "A implies B" by feeding the system data.  I think I need to get through the `train` routine like in the [supervised test case](https://github.com/IBM/LNN/blob/master/tests/learning/propositional/test_supervised_1.py)

In [53]:
# Sanity check, RHS is now TRUE and now LHS, RHS are Predicates
model = Model()
X = lnn.Variable('X')
LHS = Predicate("LHS")
RHS = Predicate("RHS")
#AB = lnn.ForAll(X, lnn.Implies(LHS(X), RHS(X), world=World.AXIOM))
AB = lnn.ForAll(X, lnn.Implies(LHS(X), RHS(X), world=World.OPEN))
#AB = lnn.ForAll(X, lnn.Implies(LHS(X), RHS(X), world=World.CLOSED))
model.add_knowledge(AB)
model.add_data({
    LHS: {'x': TRUE, 'x2': (0.8, 1.0)},
    RHS: {'y': TRUE, 'y2': (0.1, 0.4)}
})
model.train(direction=Direction.UPWARD, losses=[Loss.SUPERVISED, Loss.CONTRADICTION])
model.print(params=True)
bias = AB.params("bias")
print(AB.state())


***************************************************************************
                                LNN Model

AXIOM ForAll: (∀0, (LHS(0) → RHS(0)))                       TRUE (1.0, 1.0)
params  α: 1.0,  β: 1.0,  w: [1. 1. 1. 1.]
OPEN Implies: (LHS(0) → RHS(0)) 
params  α: 1.0,  β: 1.0,  w: [1. 1.]
'x'                                                         TRUE (1.0, 1.0)
'y'                                                         TRUE (1.0, 1.0)
'x2'                                                        TRUE (1.0, 1.0)
'y2'                                                        TRUE (1.0, 1.0)

OPEN Predicate: RHS 
params  α: 1.0
'y'                                                         TRUE (1.0, 1.0)
'y2'                                                APPROX_FALSE (0.1, 0.4)
'x'                                                         TRUE (1.0, 1.0)
'x2'                                                 APPROX_TRUE (0.8, 1.0)

OPEN Predicate: LHS 
params  α: 1.0
'x'      